## Zomato Bangalore Restaurants Prediction
The basic idea of analyzing the Zomato dataset is to get a fair idea about the factors affecting the establishment
of different types of restaurant at different places in Bengalur.

In [1]:
DF_PATH = '../data/raw/zomato.csv'
DF_SAVE_PATH = '../data/processed/ML_zomato_processed.csv'

### Import libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

### Read dataframe

In [3]:
df = pd.read_csv(DF_PATH)
df.sample(3)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
31392,https://www.zomato.com/bangalore/daawat-restau...,"Maruti Plaza, 340, 1st B Main Road, Koramangal...",Daawat Restaurant Bar & Pub,Yes,No,3.5 /5,9,+91 9964849987\r\n+91 9743028415,Koramangala 7th Block,"Bar, Casual Dining",NaN,"Finger Food, North Indian, Seafood, Mangalorea...",700,"[('Rated 5.0', 'RATED\n The place is very sma...",[],Dine-out,Koramangala 5th Block
21004,https://www.zomato.com/bangalore/90s-cafe-jp-n...,"120, 9th Cross Road, Jeewan Sathi Colony, 1st ...",90's Cafe,No,No,NEW,0,+91 9739500555,JP Nagar,Quick Bites,NaN,"Fast Food, Burger",500,[],[],Desserts,Jayanagar
38794,https://www.zomato.com/bangalore/the-chariot-b...,"30, Residency Road, Brigade Road, Bangalore",The Chariot,Yes,No,2.9 /5,50,+91 9900227802\n080 25597419,Brigade Road,Quick Bites,"Masala Dosa, Panneer Butter Masala, Thali","South Indian, North Indian, Chinese",350,"[('Rated 4.0', 'RATED\n The food is good. I h...","['Dal Tadka', 'Kajoo Masala', 'Ghee Rice', 'Ve...",Dine-out,Lavelle Road


**To do list:**
- Drop unneeded features: (name, url, address, phone, dish_liked, reviews_list, menu_item, votes).
- Fix (rate, approx_cost) features problems.
- Check null values and handle it.
- Engineer features (cuisines, rest_type, location).
- Create Target Feature.
- Remove duplicated rows.
- Save the prepared data into csv file.

---------

### Drop unneeded features: (url, address, phone, dish_liked, reviews_list, menu_item).

In [4]:
cols_to_drop = ['name', 'url', 'address', 'phone', 'dish_liked', 'reviews_list', 'menu_item', 'votes']
df.drop(cols_to_drop, axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   online_order                 51717 non-null  object
 1   book_table                   51717 non-null  object
 2   rate                         43942 non-null  object
 3   location                     51696 non-null  object
 4   rest_type                    51490 non-null  object
 5   cuisines                     51672 non-null  object
 6   approx_cost(for two people)  51371 non-null  object
 7   listed_in(type)              51717 non-null  object
 8   listed_in(city)              51717 non-null  object
dtypes: object(9)
memory usage: 3.6+ MB


### Fix (rate, approx_cost) features problems.

In [5]:
def fix_rate(rate):
    try:
        return float(rate[0:3])
    except:
        return np.nan

In [6]:
df['rate'].apply(fix_rate).unique()

array([4.1, 3.8, 3.7, 3.6, 4.6, 4. , 4.2, 3.9, 3.1, 3. , 3.2, 3.3, 2.8,
       4.4, 4.3, nan, 2.9, 3.5, 2.6, 3.4, 4.5, 2.5, 2.7, 4.7, 2.4, 2.2,
       2.3, 4.8, 4.9, 2.1, 2. , 1.8])

In [7]:
df['rate'] = df['rate'].apply(fix_rate)

In [8]:
df['rate'].unique()

array([4.1, 3.8, 3.7, 3.6, 4.6, 4. , 4.2, 3.9, 3.1, 3. , 3.2, 3.3, 2.8,
       4.4, 4.3, nan, 2.9, 3.5, 2.6, 3.4, 4.5, 2.5, 2.7, 4.7, 2.4, 2.2,
       2.3, 4.8, 4.9, 2.1, 2. , 1.8])

In [9]:
df['approx_cost(for two people)'].astype(str).apply(lambda c : float(c.replace(",", ""))).unique()

array([ 800.,  300.,  600.,  700.,  550.,  500.,  450.,  650.,  400.,
        900.,  200.,  750.,  150.,  850.,  100., 1200.,  350.,  250.,
        950., 1000., 1500., 1300.,  199.,   80., 1100.,  160., 1600.,
        230.,  130.,   50.,  190., 1700.,   nan, 1400.,  180., 1350.,
       2200., 2000., 1800., 1900.,  330., 2500., 2100., 3000., 2800.,
       3400.,   40., 1250., 3500., 4000., 2400., 2600.,  120., 1450.,
        469.,   70., 3200.,   60.,  560.,  240.,  360., 6000., 1050.,
       2300., 4100., 5000., 3700., 1650., 2700., 4500.,  140.])

In [10]:
df['approx_cost(for two people)'] = df['approx_cost(for two people)'].astype(str).apply(lambda c : float(c.replace(",", "")))

In [11]:
df['approx_cost(for two people)'].unique()

array([ 800.,  300.,  600.,  700.,  550.,  500.,  450.,  650.,  400.,
        900.,  200.,  750.,  150.,  850.,  100., 1200.,  350.,  250.,
        950., 1000., 1500., 1300.,  199.,   80., 1100.,  160., 1600.,
        230.,  130.,   50.,  190., 1700.,   nan, 1400.,  180., 1350.,
       2200., 2000., 1800., 1900.,  330., 2500., 2100., 3000., 2800.,
       3400.,   40., 1250., 3500., 4000., 2400., 2600.,  120., 1450.,
        469.,   70., 3200.,   60.,  560.,  240.,  360., 6000., 1050.,
       2300., 4100., 5000., 3700., 1650., 2700., 4500.,  140.])

### Check null values and handle it.

In [12]:
df.isna().mean() * 100

online_order                    0.000000
book_table                      0.000000
rate                           19.436549
location                        0.040606
rest_type                       0.438927
cuisines                        0.087012
approx_cost(for two people)     0.669026
listed_in(type)                 0.000000
listed_in(city)                 0.000000
dtype: float64

> **Note:** The null values count are not many regarding the count of dataset, so we will drop these rows.

In [13]:
df.dropna(axis = 0, inplace = True)

In [14]:
df.isna().mean() * 100

online_order                   0.0
book_table                     0.0
rate                           0.0
location                       0.0
rest_type                      0.0
cuisines                       0.0
approx_cost(for two people)    0.0
listed_in(type)                0.0
listed_in(city)                0.0
dtype: float64

### Engineer features (cuisines, rest_type, location).

In [15]:
df['rest_type_counts'] = df['rest_type'].astype(str).apply(lambda c : len(c.split(',')))

In [16]:
df['cuisines_counts'] = df['cuisines'].astype(str).apply(lambda c : len(c.split(',')))

#### Drop 2 features (rest_type, cuisines)

In [17]:
df.drop(['rest_type', 'cuisines'], axis = 1, inplace = True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41263 entries, 0 to 51716
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   online_order                 41263 non-null  object 
 1   book_table                   41263 non-null  object 
 2   rate                         41263 non-null  float64
 3   location                     41263 non-null  object 
 4   approx_cost(for two people)  41263 non-null  float64
 5   listed_in(type)              41263 non-null  object 
 6   listed_in(city)              41263 non-null  object 
 7   rest_type_counts             41263 non-null  int64  
 8   cuisines_counts              41263 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 3.1+ MB


In [19]:
df_locations_perc = df['location'].value_counts(normalize=True) * 100
df_locations_perc

location
BTM                      9.400674
Koramangala 5th Block    5.566730
HSR                      4.829993
Indiranagar              4.362262
JP Nagar                 4.144149
                           ...   
Yelahanka                0.009694
West Bangalore           0.007270
Rajarajeshwari Nagar     0.004847
Nagarbhavi               0.002423
Peenya                   0.002423
Name: proportion, Length: 92, dtype: float64

In [20]:
desired_locations = df_locations_perc[df_locations_perc>0.5]
desired_locations

location
BTM                      9.400674
Koramangala 5th Block    5.566730
HSR                      4.829993
Indiranagar              4.362262
JP Nagar                 4.144149
Jayanagar                3.959964
Whitefield               3.800015
Marathahalli             3.417105
Bannerghatta Road        2.971185
Koramangala 7th Block    2.556770
Koramangala 6th Block    2.554347
Brigade Road             2.549500
Bellandur                2.416208
Sarjapur Road            2.069651
Koramangala 1st Block    2.064804
Ulsoor                   2.057533
Koramangala 4th Block    2.038146
Electronic City          2.021181
MG Road                  1.921819
Banashankari             1.803068
Kalyan Nagar             1.677047
Malleshwaram             1.553450
Residency Road           1.463781
Richmond Road            1.463781
Basavanagudi             1.441970
Frazer Town              1.371689
Church Street            1.323219
Brookefield              1.315949
New BEL Road             1.226280
Kamma

In [21]:
def fix_location(r):
    if r in desired_locations:
        return r
    else:
        return 'other'

In [22]:
df['location'] = df['location'].apply(fix_location)
df['location'].value_counts(normalize=True) * 100

location
BTM                      9.400674
other                    7.680004
Koramangala 5th Block    5.566730
HSR                      4.829993
Indiranagar              4.362262
JP Nagar                 4.144149
Jayanagar                3.959964
Whitefield               3.800015
Marathahalli             3.417105
Bannerghatta Road        2.971185
Koramangala 7th Block    2.556770
Koramangala 6th Block    2.554347
Brigade Road             2.549500
Bellandur                2.416208
Sarjapur Road            2.069651
Koramangala 1st Block    2.064804
Ulsoor                   2.057533
Koramangala 4th Block    2.038146
Electronic City          2.021181
MG Road                  1.921819
Banashankari             1.803068
Kalyan Nagar             1.677047
Malleshwaram             1.553450
Residency Road           1.463781
Richmond Road            1.463781
Basavanagudi             1.441970
Frazer Town              1.371689
Church Street            1.323219
Brookefield              1.315949
New B

### Create Target Feature.

In [23]:
df['rate'].describe()

count    41263.000000
mean         3.702011
std          0.439985
min          1.800000
25%          3.400000
50%          3.700000
75%          4.000000
max          4.900000
Name: rate, dtype: float64

In [24]:
def create_target(rate):
    try:
        if rate >= 3.75:
            return  1
        else:
            return 0
    except:
        return np.nan

In [25]:
df['success'] = df['rate'].apply(create_target)
df.drop('rate', axis = 1, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41263 entries, 0 to 51716
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   online_order                 41263 non-null  object 
 1   book_table                   41263 non-null  object 
 2   location                     41263 non-null  object 
 3   approx_cost(for two people)  41263 non-null  float64
 4   listed_in(type)              41263 non-null  object 
 5   listed_in(city)              41263 non-null  object 
 6   rest_type_counts             41263 non-null  int64  
 7   cuisines_counts              41263 non-null  int64  
 8   success                      41263 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 3.1+ MB


In [26]:
df['success'].value_counts()

success
0    21185
1    20078
Name: count, dtype: int64

### Remove duplicated rows.

In [27]:
df.duplicated().sum()

9250

In [28]:
df.drop_duplicates(inplace = True)

In [29]:
df.duplicated().sum()

0

### Save the prepared data into csv file.

In [30]:
df.to_csv(DF_SAVE_PATH, index = False)